# Market Simulation

<b>Description:</b> This notebooks runs the market clearing simulation given multiple bidder objects to measure performance.

In [9]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.MarketSimulator import *

## Load Energy Storage and Price Data

In [39]:
# load storage data
ENERGY_STORAGE_DATA_PATH = (
    "./src/CAISO-EnergyStorage/src/data/ES_BIDS/CAISO_ES_BIDS.parquet"
)
STORAGE_DF = pd.read_parquet(ENERGY_STORAGE_DATA_PATH)
STORAGE_DF = STORAGE_DF.ffill()

# load day-ahead and real-time prices
DAP = pd.read_csv("./data/CAISO_DAP.csv")
RTP = pd.read_csv("./data/CAISO_RTP.csv")

# read datetime and drop duplicate data
RTP["Date"] = pd.to_datetime(RTP["Date"], format="%m/%d/%Y %I:%M:%S %p")
RTP = RTP.drop_duplicates(subset=["Date", "hub"])
DAP["Date"] = pd.to_datetime(DAP["Date"], format="%m/%d/%Y %I:%M:%S %p")
DAP = DAP.drop_duplicates(subset=["Date", "zone"])

# pivot data
DAP_pivoted = DAP.pivot(index="Date", columns="zone", values="price")
RTP_pivoted = RTP.pivot(index="Date", columns="hub", values="price")

# rename 'Date' column to 'ts'
DAP_pivoted.index.names = ["ts"]
RTP_pivoted.index.names = ["ts"]

# merge dataframes on index
CAISO_PRICES = pd.merge(DAP_pivoted, RTP_pivoted, on=["ts"], how="outer")
CAISO_PRICES = CAISO_PRICES.ffill().reset_index()

# rename ts to datetime and set as index
CAISO_PRICES = CAISO_PRICES.rename(columns={"ts": "datetime"})
CAISO_PRICES = CAISO_PRICES.set_index("datetime")

# merge STORAGE_DF and CAISO_PRICES on datetime
storage_df = pd.merge(CAISO_PRICES, STORAGE_DF, on=["datetime"], how="outer").dropna()
storage_df

,PGAE,SCE,SDGE,VEA,TH_NP15,TH_SP15,TH_ZP26,tot_energy_ifm,tot_energy_ruc,tot_energy_rtpd,...,bid_rtpd_neg_2,bid_rtpd_neg_3,bid_rtpd_neg_4,bid_rtpd_neg_5,bid_rtpd_neg_6,bid_rtpd_neg_7,bid_rtpd_neg_8,bid_rtpd_neg_9,bid_rtpd_neg_10,bid_rtpd_neg_11
datetime,,,,,,,,,,,,,,,,,,,,,
2023-06-01 00:00:00,29.36043,30.02044,30.78214,29.14621,25.25711,26.13978,25.54956,-13.0,-13.0,248.0,...,-65.0,-894.7,-293.6,-1033.0,-52.16,-1.00,0.00,0.0,0.00,0.0
2023-06-01 00:05:00,29.36043,30.02044,30.78214,29.14621,25.72560,26.62464,26.02347,-13.0,-13.0,248.0,...,-65.0,-894.7,-293.6,-1033.0,-52.16,-1.00,0.00,0.0,0.00,0.0
2023-06-01 00:10:00,29.36043,30.02044,30.78214,29.14621,25.46748,26.35750,25.76237,-13.0,-13.0,248.0,...,-65.0,-894.7,-293.6,-1033.0,-52.16,-1.00,0.00,0.0,0.00,0.0
2023-06-01 00:15:00,29.36043,30.02044,30.78214,29.14621,25.61265,26.28328,25.85139,-13.0,-13.0,349.0,...,-65.0,-894.7,-293.6,-1033.0,-52.16,-1.00,0.00,0.0,0.00,0.0
2023-06-01 00:20:00,29.36043,30.02044,30.78214,29.14621,26.53341,27.22815,26.78074,-13.0,-13.0,349.0,...,-65.0,-894.7,-293.6,-1033.0,-52.16,-1.00,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-01 23:35:00,52.87647,50.74752,50.85212,49.38652,42.70535,43.56328,43.57238,112.0,112.0,-447.0,...,0.0,-512.3,-237.0,-1072.0,-3365.00,-93.23,-34.88,0.0,-18.88,0.0
2024-10-01 23:40:00,52.87647,50.74752,50.85212,49.38652,41.98404,42.85560,42.86452,112.0,112.0,-447.0,...,0.0,-512.3,-237.0,-1072.0,-3365.00,-93.23,-34.88,0.0,-18.88,0.0
2024-10-01 23:45:00,52.87647,50.74752,50.85212,49.38652,41.19628,42.09376,42.08496,112.0,112.0,-684.0,...,0.0,-512.3,-237.0,-1072.0,-3365.00,-93.23,-34.88,0.0,-18.88,0.0
